# INTRODUCTION

## Contexte du Projet

## Mise en place de PySpark

In [27]:
# Installation de pyspark
%pip install pyspark


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


IMPORTANT : PENSEZ A INSTALLER JAVA - winget install --exact --id EclipseAdoptium.Temurin.11.JDK

Fix pour python 3.12 qui ne contient plus la librairie distutils

In [28]:
import sys
try:
    import distutils
except ModuleNotFoundError:
    import setuptools._distutils as distutils
    sys.modules["distutils"] = distutils

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MoviesRatings") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


## Import et traitement des datasets

In [30]:
# Import des datasets ratings.csv & movies.csv
ratings = spark.read.csv("ml-latest-small/ratings.csv", header=True, inferSchema=True)
movies = spark.read.csv("ml-latest-small/movies.csv", header=True, inferSchema=True)


In [31]:
# On Affiche les les 10 premières lignes des deux datasets
ratings.show(10)
movies.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|

In [32]:
# On affiche le schéma des deux datasets
ratings.printSchema()
movies.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [33]:
# On regarde le nombre de lignes de chaque dataset
print("Nombre de lignes dans ratings : ", ratings.count())
print("Nombre de lignes dans movies : ", movies.count())

Nombre de lignes dans ratings :  100836
Nombre de lignes dans movies :  9742


### Imports PySpark

In [34]:
from pyspark.sql.functions import isnan, when, count, col, avg, lit, explode, split, row_number
from pyspark.sql.window import Window
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import collect_list

In [35]:
# On regarde le nombre de valeurs manquantes dans chaque dataset

ratings.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ratings.columns]).show()
movies.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in movies.columns]).show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     0|      0|     0|        0|
+------+-------+------+---------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|      0|    0|     0|
+-------+-----+------+



In [36]:
# On regarde le nombre de valeurs distinctes dans chaque dataset
ratings.agg(*(count(col(c)).alias(c) for c in ratings.columns)).show()
movies.agg(*(count(col(c)).alias(c) for c in movies.columns)).show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|100836| 100836|100836|   100836|
+------+-------+------+---------+

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|   9742| 9742|  9742|
+-------+-----+------+



In [37]:
# On regarde les statistiques descriptives des colonnes rating et timestamp du dataset ratings
ratings.describe(["rating", "timestamp"]).show()


+-------+------------------+--------------------+
|summary|            rating|           timestamp|
+-------+------------------+--------------------+
|  count|            100836|              100836|
|   mean| 3.501556983616962|1.2059460873684695E9|
| stddev|1.0425292390606342|2.1626103599513078E8|
|    min|               0.5|           828124615|
|    max|               5.0|          1537799250|
+-------+------------------+--------------------+



In [38]:
# Quels sont les films les mieux notés en moyenne 

ratings.groupBy("movieId").agg(avg("rating").alias("avg_rating")).orderBy("avg_rating", ascending=False).show()

+-------+----------+
|movieId|avg_rating|
+-------+----------+
|  26350|       5.0|
|   3795|       5.0|
|  25887|       5.0|
| 157775|       5.0|
|    633|       5.0|
|  33138|       5.0|
|  67618|       5.0|
|    876|       5.0|
|    496|       5.0|
|  27373|       5.0|
| 113829|       5.0|
|  53578|       5.0|
| 152711|       5.0|
| 118894|       5.0|
|     53|       5.0|
| 160644|       5.0|
|    148|       5.0|
|   8911|       5.0|
| 147300|       5.0|
|  84273|       5.0|
+-------+----------+
only showing top 20 rows



In [39]:
# Quels sont les films les plus populaires 
ratings.groupBy("movieId").count().orderBy("count", ascending=False).show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [40]:
# Quels sont les 10 films les mieux notés en moyenne avec plus de 100/200 & 300 notes

# Jointure commune entre ratings et movies
ratings_with_titles = ratings.join(movies, on="movieId", how="inner")

# Fonction pour obtenir les films mieux notés avec un seuil minimal de votes
def top_movies_by_avg(threshold, limit=10):
    return (
        ratings_with_titles
        .groupBy("movieId", "title")
        .agg(avg("rating").alias("avg_rating"), count("rating").alias("count_rating"))
        .filter(col("count_rating") > threshold)
        .orderBy(col("avg_rating").desc())
        .select("title", "avg_rating", "count_rating")
        .limit(limit)
    )

top_movies_by_avg(100).show(10, False)
top_movies_by_avg(200).show(10, False)
top_movies_by_avg(300).show(10, False)

+-----------------------------------------+-----------------+------------+
|title                                    |avg_rating       |count_rating|
+-----------------------------------------+-----------------+------------+
|Shawshank Redemption, The (1994)         |4.429022082018927|317         |
|Godfather, The (1972)                    |4.2890625        |192         |
|Fight Club (1999)                        |4.272935779816514|218         |
|Godfather: Part II, The (1974)           |4.25968992248062 |129         |
|Departed, The (2006)                     |4.252336448598131|107         |
|Goodfellas (1990)                        |4.25             |126         |
|Dark Knight, The (2008)                  |4.238255033557047|149         |
|Usual Suspects, The (1995)               |4.237745098039215|204         |
|Princess Bride, The (1987)               |4.232394366197183|142         |
|Star Wars: Episode IV - A New Hope (1977)|4.231075697211155|251         |
+------------------------

In [41]:
# Quels genres de films sont les plus populaires ? (nb de notes) & Top 5 des films les plus populaires par genre

def most_popular_movies(limit=30):
    return (
        ratings.groupBy("movieId")
               .agg(count("rating").alias("count_rating"))
               .join(movies, "movieId")
               .orderBy(col("count_rating").desc())
               .limit(limit)
    )

def top5_by_genre():
    movies_with_genre = movies.withColumn("genre", explode(split(col("genres"), "\\|")))
    movies_ratings = ratings.join(movies_with_genre, "movieId")
    movies_stats = movies_ratings.groupBy("title", "genre") \
        .agg(count("rating").alias("count_rating"), avg("rating").alias("avg_rating"))
    windowSpec = Window.partitionBy("genre").orderBy(col("count_rating").desc())
    return movies_stats.withColumn("rank", row_number().over(windowSpec)) \
        .filter(col("rank") <= 5) \
        .orderBy("genre", "rank")

def display_top5_by_genre(limit_per_genre=5):
    df = top5_by_genre()
    # Récupérer la liste des genres distincts
    genres = [row["genre"] for row in df.select("genre").distinct().collect()]
    for genre in sorted(genres):
        print(f"Genre: {genre}")
        df.filter(col("genre") == genre).orderBy("rank").show(limit_per_genre, False)

# Utilisation :

print("Most Popular Movies:")
most_popular_movies().show(30, False)

print("\nTop 5 Movies by Genre:")
display_top5_by_genre()

Most Popular Movies:
+-------+------------+------------------------------------------------------------------------------+-------------------------------------------+
|movieId|count_rating|title                                                                         |genres                                     |
+-------+------------+------------------------------------------------------------------------------+-------------------------------------------+
|356    |329         |Forrest Gump (1994)                                                           |Comedy|Drama|Romance|War                   |
|318    |317         |Shawshank Redemption, The (1994)                                              |Crime|Drama                                |
|296    |307         |Pulp Fiction (1994)                                                           |Comedy|Crime|Drama|Thriller                |
|593    |279         |Silence of the Lambs, The (1991)                                              |Cr

## Modelisation avec Spark MLlib : ALS

In [54]:
# S'assurer que les colonnes ont le bon type
ratings = ratings.withColumn("userId", col("userId").cast("integer")) \
                 .withColumn("movieId", col("movieId").cast("integer")) \
                 .withColumn("rating", col("rating").cast("float"))

# Diviser en 80% training et 20% test
(training, test) = ratings.randomSplit([0.8, 0.2])

In [101]:
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=15,           # Nombre d'itérations
    regParam=0.185,         # Paramètre de régularisation
    rank=15,              # Dimension des facteurs latents
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating",
    coldStartStrategy="drop"  # Pour ignorer les prédictions NaN lors de l'évaluation
)

# Entraîner le modèle sur le jeu de données d'entraînement
model = als.fit(training)

In [102]:
# Prédire les ratings sur le jeu de test
predictions = model.transform(test)

# Calcul du RMSE
evaluator_rmse = RegressionEvaluator(metricName="rmse",
                                       labelCol="rating",
                                       predictionCol="prediction")
rmse = evaluator_rmse.evaluate(predictions)
print(f"RMSE: {rmse:.4f}")

# Calcul du MAE
evaluator_mae = RegressionEvaluator(metricName="mae",
                                      labelCol="rating",
                                      predictionCol="prediction")
mae = evaluator_mae.evaluate(predictions)
print(f"MAE: {mae:.4f}")

RMSE: 0.8822
MAE: 0.6871


In [78]:
# Générer des recommandations pour tous les utilisateurs
userRecs = model.recommendForAllUsers(10)
userRecs.show(10, False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                                         |
+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |[{170355, 6.0245786}, {132333, 5.905312}, {33649, 5.7328176}, {60943, 5.7129793}, {59018, 5.7129793}, {7841, 5.7129793}, {25947, 5.65016}, {26326, 5.5881605}, {67618, 5.55364}, {65642, 5.501053}]     |
|2     |[{67618, 4.9944696}, {170355, 4.719432}, {33649, 4.7128296}, {77846, 4.6475163}, {25906, 4.6475163}, {184245, 4.596199}, {179135, 4.596199}, {171495, 4.

In [79]:
# Générer les recommandations pour un utilisateur spécifique
userRecs = model.recommendForUserSubset(ratings.filter(col("userId") == 123), 5)
userRecs = userRecs.withColumn("rec", explode(col("recommendations"))) \
                   .select(col("userId"), col("rec.movieId"), col("rec.rating")) \
                   .join(movies, "movieId", "left") \
                   .select("userId", "title", "rating")

userRecs.show(truncate=False)

+------+---------------------+---------+
|userId|title                |rating   |
+------+---------------------+---------+
|123   |Mulholland Dr. (1999)|5.1597776|
|123   |Saving Face (2004)   |4.9754558|
|123   |Seve (2014)          |4.9539247|
|123   |Frozen River (2008)  |4.939108 |
|123   |Visitor, The (2007)  |4.939108 |
+------+---------------------+---------+



## Recommandation basée sur le contenu

## Recommandation basée sur les proximités utilisateurs (KNN)

## Evaluation des approches de recommandation